In [9]:
!git clone https://github.com/going-digital/ml_sdf.git

fatal: destination path 'ml_sdf' already exists and is not an empty directory.


In [10]:
!pip install mesh_to_sdf tf_siren
import numpy as np
import tensorflow as tf
from mesh_to_sdf import get_surface_point_cloud
from mesh_to_sdf.utils import sample_uniform_points_in_unit_sphere
import trimesh
from tf_siren import SIRENModel
import re

In [11]:
seed = 1234
np.random.seed(seed)
tf.random.set_seed(seed)

In [12]:
def SDFFitting(filename, samples):
    surface_samples = int(np.floor(0.7 * samples))
    volume_samples = samples - surface_samples
    mesh = trimesh.load(filename)
    surface_point_cloud = get_surface_point_cloud(mesh, surface_point_method='sample')
    coords1, samples1 = surface_point_cloud.sample_sdf_near_surface(surface_samples, use_scans=False, sign_method='normal')
    coords2 = sample_uniform_points_in_unit_sphere(volume_samples)
    samples2 = surface_point_cloud.get_sdf_in_batches(coords2, use_depth_buffer=False)
    coords = np.concatenate([coords1,  coords2])
    samples = np.concatenate([samples1, samples2])
    return tf.data.Dataset.from_tensor_slices((coords, samples))

In [13]:
sdf = SDFFitting("ml_sdf/bunny2.obj",256*256*4)

In [14]:
train_dataset = sdf.shuffle(10000).batch(65536).cache()
model = SIRENModel(
    units=12,   # Model enlarges with the square of this
    num_layers=3, # Model enlarges proportionally to this
    final_units=1, # 1 channel output - distance
    final_activation='linear',
    w0_initial=15., # Metaparameter
    w0=20.0,
)
_ = model(tf.zeros([1,3]))

num_steps = 10000 # 40000
learning_rate = tf.keras.optimizers.schedules.PolynomialDecay(0.005, decay_steps=num_steps, end_learning_rate=0.0005, power=2.0)
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
#loss = tf.keras.losses.MeanAbsoluteError(reduction=tf.keras.losses.Reduction.NONE)
loss = tf.keras.losses.MeanSquaredError(reduction=tf.keras.losses.Reduction.NONE)
#loss = tf.keras.losses.MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.NONE)
model.compile(optimizer, loss=loss)
model.fit(train_dataset, epochs=num_steps, verbose=2)

Streaming output truncated to the last 5000 lines.
Epoch 7501/10000
4/4 - 0s - loss: 1.1796e-04
Epoch 7502/10000
4/4 - 0s - loss: 1.1289e-04
Epoch 7503/10000
4/4 - 0s - loss: 1.0523e-04
Epoch 7504/10000
4/4 - 0s - loss: 1.0219e-04
Epoch 7505/10000
4/4 - 0s - loss: 9.6687e-05
Epoch 7506/10000
4/4 - 0s - loss: 9.9487e-05
Epoch 7507/10000
4/4 - 0s - loss: 1.0147e-04
Epoch 7508/10000
4/4 - 0s - loss: 1.0564e-04
Epoch 7509/10000
4/4 - 0s - loss: 1.0847e-04
Epoch 7510/10000
4/4 - 0s - loss: 1.1249e-04
Epoch 7511/10000
4/4 - 0s - loss: 1.1478e-04
Epoch 7512/10000
4/4 - 0s - loss: 1.2124e-04
Epoch 7513/10000
4/4 - 0s - loss: 1.2235e-04
Epoch 7514/10000
4/4 - 0s - loss: 1.2569e-04
Epoch 7515/10000
4/4 - 0s - loss: 1.1989e-04
Epoch 7516/10000
4/4 - 0s - loss: 1.2807e-04
Epoch 7517/10000
4/4 - 0s - loss: 1.2021e-04
Epoch 7518/10000
4/4 - 0s - loss: 1.3622e-04
Epoch 7519/10000
4/4 - 0s - loss: 1.2984e-04
Epoch 7520/10000
4/4 - 0s - loss: 1.4050e-04
Epoch 7521/10000
4/4 - 0s - loss: 1.3536e-04
Epoc

In [15]:
def model_to_shadertoy(model):
    out = "float scene(vec3 p) {\n"
    out += "  if (length(p) > 1.) return length(p)-.8;\n"
    out += "  vec4 "
    vec4_defs = ["x=vec4(p,1)"]
    
    layers = len(model.weights) // 2
    assert(model.weights[0].shape[0]==3) # Input dimensionality of 3
    assert(model.weights[-1].shape==1) # Output dimensionality of 1
    
    def vec4(n):
        return 'vec4(' + ','.join(['{0:.2f}'.format(i) for i in n.flatten()]) + ')'

    def mat4(n, transpose=False):
        if transpose: n = np.transpose(n)
        return 'mat4(' + ','.join(['{0:.2f}'.format(i) for i in n.flatten()]) + ')'

    def vname(layer, chunk):
        return "f%d%d" % (layer, chunk)

    # First layer
    # Input vector includes 3 axes and a '1' term, to integrate bias.
    input_dim, output_dim = model.weights[0].shape
    assert(output_dim % 4 == 0)
    w0 = model.siren_layers.layers[0].w0
    for chunk in range(output_dim // 4):
        mat = np.concatenate([
            w0*model.weights[0][2, chunk*4:4+chunk*4],
            w0*model.weights[0][0, chunk*4:4+chunk*4],
            w0*model.weights[0][1, chunk*4:4+chunk*4],
            w0*model.weights[1][chunk*4:4+chunk*4]
        ]).reshape(4,4)
        vec4_defs.append(
            '{}=sin(x*{})'.format(
                vname(0,chunk),
                mat4(mat, transpose=True)
            )
        )
    
    # Hidden layers
    for layer in range(1, layers-1):
        w0 = model.siren_layers.layers[layer].w0
        input_dim, output_dim = model.weights[2*layer].shape
        assert(input_dim % 4 == 0)
        assert(output_dim % 4 == 0)
        for out_chunk in range(output_dim // 4):
            elements = []
            for in_chunk in range(input_dim // 4):
                elements.append(
                    mat4(w0*model.weights[2*layer][in_chunk*4:4+in_chunk*4, out_chunk*4:4+out_chunk*4].numpy(), transpose=False)
                    + '*'
                    + vname(layer-1, in_chunk)
                )
            elements.append(
                vec4(w0*model.weights[2*layer+1][out_chunk*4:4+out_chunk*4].numpy())
            )
            vec4_defs.append(
                '{}=sin({})'.format(
                    vname(layer, out_chunk),
                    '+'.join(elements)
                )
            )
        
    # Build into first statement
    out += ',\n    '.join(vec4_defs) + ";\n"

    # Output layer is separate return statement
    layer = layers - 1
    elements = []
    input_dim, output_dim = model.weights[2*layer].shape
    assert(input_dim % 4 == 0)
    assert(output_dim == 1)
    w0 = model.siren_layers.layers[-1].w0
    for in_chunk in range(input_dim // 4):
        elements.append(
            "dot({},{})".format(
                vec4(model.weights[2*layer][4*in_chunk:4+4*in_chunk].numpy()),
                vname(layer - 1, in_chunk)
            )
        )
    elements.append("{0:.2f}".format(model.weights[2*layer+1][0]))
    out += "  return " + "+".join(elements) + ";\n"
    out += "}\n"

    # Simplifying substitutions
    out = re.sub(r"(\d+\.\d*)0+\b", r"\1", out) # Remove trailing zeros eg. 1.0 => 1.
    out = re.sub(r"\b(\.\d+)0+\b", r"\1", out) # Remove trailing zeros eg. .60 => .6
    out = re.sub(r"\b0(\.\d+)\b", r"\1", out) # Remove leading zeros eg. 0.5 => .5
    out = re.sub(r"-\.0+\b", r".0", out) # Make all zeros positive eg. -.0 => .0
    out = re.sub(r"\+-", r"-", out) # Change +-1. into -1.

    return out

In [16]:
print(model_to_shadertoy(model))

float scene(vec3 p) {
  if (length(p) > 1.) return length(p)-.8;
  vec4 x=vec4(p,1),
    f00=sin(x*mat4(-1.74,-1.05,-1.82,2.64,-3.5,-4.23,2.91,-3.83,-.67,2.72,1.37,10.45,-1.46,-1.9,3.0,5.41)),
    f01=sin(x*mat4(-3.12,-3.3,1.11,-1.54,2.91,-2.94,.82,-9.88,.92,1.34,2.62,2.71,1.16,1.04,-1.25,-9.69)),
    f02=sin(x*mat4(3.42,-1.21,2.28,9.64,-2.02,1.15,1.83,-4.09,3.23,-.52,2.07,5.01,.52,-.6,3.35,.87)),
    f10=sin(mat4(-.99,.53,-.28,.66,.15,-.02,-.09,-.06,-.01,-.05,-.04,-.56,-.1,-.31,-.83,-.43)*f00+mat4(.15,.04,-.09,.21,.42,-.23,.02,.17,.23,.31,.6,.56,-.93,1.47,.0,-.48)*f01+mat4(.71,-.44,-.18,.43,-.6,.42,.45,.5,-.92,.35,-.72,-.44,.0,.76,-.31,-.05)*f02+vec4(-13.95,-3.14,5.36,-9.25)),
    f11=sin(mat4(-.54,-.75,-.21,-.24,-.38,.06,-.03,.55,.81,.51,-1.12,-.01,-.38,.11,-.02,.67)*f00+mat4(.11,.15,-.05,1.59,.04,-.23,-.18,-1.16,-.08,-.14,-.53,-.07,.23,-.36,-.4,-.15)*f01+mat4(.72,.28,-.18,-.61,.22,-.23,-.65,.28,.83,.41,.11,-.29,-.39,-.32,-.92,-.51)*f02+vec4(-4.81,6.95,9.26,-6.26)),
    f12=sin(mat4(